# PRICING in VINTAGE SHIRTS

In [3]:
import requests
#BeautifulSoup es una calse dentro de la libreria bs4
from bs4 import BeautifulSoup
import pandas as pd
# from pprint import pprint
# from lxml import html
# from lxml.html import fromstring
# import urllib.request
from urllib.request import urlopen
# import random
import re
# import scrapy

In [4]:
def acces_collections(collection,page_no):
    
    "This function generates a soup of the page 'x' of a filtered search based on collections"
    
    #1. We defin the url for the search based on collection and page:
    url = f"https://www.wycovintage.com/collections/{collection}?filter.p.product_type=Shirt&options%5Bprefix%5D=last&page={page_no}&sort_by=created-descending"

    #2. Reques the html code for the url mentioned
    html = requests.get(url)
    
    #3. Print the request resoponse, for visibility 
    print(html)
    
    #4. Return an objetct type BeautifulSoup with the content parsed
    return BeautifulSoup(html.content, "html.parser")

In [5]:
def last_page(soup):
    
    "This function returns the number of pages inside the same filtered shearch"
    
    #1. Find the element where pagination/index is contained
    index = soup.find("ul", {"class":"Pagination"})
    
    #2. Extract all elements inside pagination/index that represet a number page - list returned
    pags_list = index.find_all("a", {"class":"Button Pagination__Link"})
    
    #3. Print the last page, for visibility 
    print(f'Last page in pagination is nº{pags_list[-1].getText()}')
          
    #3. We acces the last element of the list = last page and retunr the text
    return pags_list[-1].getText()


### Retrieve elements for a given page - debugging

Initial workarround:

In [134]:
"""This was the original function proposed to retrieve the elements on each page"""

def page_elements(soup):
    
    #1. Generate a list of descriptios for all the elements in the page:
    desc = [i.getText().strip() for i in soup.find_all("h2", {"class":"ProductCard__Title"})]
    
    #2. Generate a list of prices for all the elements in the page:
    price = [i.getText().strip() for i in soup.find_all("span", {"class":"Price"})]
    
    #Zipping desc and price
    
    #Tranforming to a dictionaty 
    
    #Returning the dictionary
    return

We have noticed that when generatin desc and price both list don't have the same length so we get
an error while creating the dictionary as both list need to match in length

To undertand what is happening we will tranform our original function into a debugging function 
to analice depper the posible causes of this bug:


In [6]:
"""I tranformed this function into a debbug function to try to analice what is hapening"""

def debugging(soup):
    
    #1. Generate a list of descriptios for all the elements in the page:
    desc = [i.getText().strip() for i in soup.find_all("h2", {"class":"ProductCard__Title"})]
    
    #2. Generate a list of prices for all the elements in the page:
    price = [i.getText().strip() for i in soup.find_all("span", {"class":"Price"})]
    
    print(f"Descriptions list has {len(desc)} and the las one is {desc[-1]}")
    print(f"Descriptions list has {len(price)} and the las one is {price[-1]}")
    return

### Checkig for bugs: 

1. Inspectiong different pages in the same collection: 

In [8]:
soup = acces_collections("1970s",1)
debugging(soup)

<Response [200]>
Descriptions list has 40 and the las one is 1979 Led Zeppelin Rules America Shirt
Descriptions list has 41 and the las one is $68


In [9]:
soup = acces_collections("1970s",2)
debugging(soup)

<Response [200]>
Descriptions list has 40 and the las one is 1970s Woodstock Shirt
Descriptions list has 41 and the las one is $68


In [10]:
soup = acces_collections("1970s",3)
debugging(soup)

<Response [200]>
Descriptions list has 40 and the las one is 1970s Dale Pulde War Eagle Orange County International Raceway Shirt
Descriptions list has 41 and the las one is $68


1. Inspecting different collection: 

In [13]:
soup = acces_collections("1960s",1)
debugging(soup)

<Response [200]>
Descriptions list has 2 and the las one is 1968 Hardwick's Dinghy Derby Shirt
Descriptions list has 3 and the las one is $68


In [12]:
soup = acces_collections("1980s",3)
debugging(soup)

<Response [200]>
Descriptions list has 40 and the las one is 1981 Siouxsie and the Banshees Ju-Ju Queen Shirt
Descriptions list has 41 and the las one is $68


In [14]:
soup = acces_collections("1990s",7)
debugging(soup)

<Response [200]>
Descriptions list has 40 and the las one is 1993 Green Bay Packers Shirt
Descriptions list has 41 and the las one is $68


In [15]:
soup = acces_collections("2000s",4)
debugging(soup)

<Response [200]>
Descriptions list has 40 and the las one is 2000 Marilyn Manson Emperor Shirt
Descriptions list has 41 and the las one is $68


### Conclusion:

I noticed that price list allays have one more element and its always the same value: $68

To avoid this bugg and future buggs wile matching the elements we wil target first the box where all info of a product is contain and then, will extratct the informations. This way we can ensure that we are getting the corresponging desc-item correcly. 